In [1]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


### 准备数据
将数值范围很大的数据输入到神经网络中，是有问题的。网络可能会自动适应这种取值范围不同的数据，但学习肯定变得更加困难。对于这种数据，普遍采用的最佳实践是对每个特征做**标准化**，即**对输入数据的每个特征(输入数据矩阵中的列)，减去特征平均值，再除以标准差，这样得到的特征平均值为0，标准差为1。**使用Numpy可以很容易实现标准化。

**用于测试数据标准化的平均值和标准差都是在训练数据上计算得到的。**
**在工作流程中，不能使用在测试数据计算得到的任何结果，即使是数据标准化这么简单的事情也不行。**

In [14]:
# 数据标准化
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean  # 采用的是训练集的平均值
test_data /= std  # 采用的是训练集的标准差

### 构建网络
一般来说，训练数据越少，过拟合会越严重，而较小的网络可以降低过拟合

In [15]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

**网络的最后一层只有一个单元，没有激活，是一个线性层。这是标量回归（标量回归是预测单一连续值的回归）
的典型设置。添加激活函数将会限制输出范围。  
例如，如果向最后一层添加sigmod激活函数，网络只能预测0～1
范围内的值，这里最后一层是纯线性的，所以网络可以预测任意范围内的值。  
注：  
1、编译网络时用的是mse损失函数，即均方误差(MSE, mean squared error)，预测值与目标值之差的平方。
这是回归问题常用的损失函数。
2、训练过程中监控了一个新指标：平均绝对误差(MAE, mean absolute error)。它是预测值与标准值之差的绝对值。
比如这个问题的MAE等于0.5，就表示预测的房价与实际价格平均相差500美元。**

### 利用K折验证
适合应用到数据集小的情况
将数据氛围k个分区(k通常为4或5)，实例化k个相同的模型，将每个模型在k-1个分区上训练，并在剩下的一个分区上进行评估。

In [23]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold %', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0
    )
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0
    )
    
    model = build_model()
    model.fit(
        partial_train_data, partial_train_targets,
        epochs=num_epochs,
        batch_size=1,
        verbose=0
    )  # 静默模式，verbose=0
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold % 0
processing fold % 1
processing fold % 2
processing fold % 3


In [24]:
all_scores

[2.118621843876225, 2.3930334383898444, 2.886435895863146, 2.3571436830086285]

In [25]:
np.mean(all_scores)

2.438808715284461